# Feature Engineering Exercises


2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [1]:
from pydataset import data
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing

1. Load the tips dataset.

    a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.
    
    b. Create a column named price_per_person. This should be the total bill divided by the party size.
    
    c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?
    
    d. Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?
    
    e. Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage. What are they?
    
    f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?




In [2]:
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df["tip_percentage"] = df.tip / df.total_bill
df["price_per_person"] = df.total_bill / df.size